<a href="https://colab.research.google.com/github/Carlos-G-Santos/Materiais_ML_IFUSP/blob/main/C%C3%B3pia_de_PreProcessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aula sobre pré processamento de dados
Nasta aula, vamos praticar alguns conceitos sobre o pré processamento de dados para aplicações de aprendizado de máquina.
Vamos ver o carregamento dos dados, o tratamento de dados ausentes, e por último, algumas correções de escala.
Este exemplo é meramente ilustrativo!

## Importando bibliotecas

In [ ]:
import pandas as pd
import numpy as np

## Carregando base de dados
Usando a função de leitura de arquivos Excel da biblioteca Pandas

In [ ]:
dados = pd.read_excel('Dados_PreProcessing.xlsx', index_col=None)

Para ter uma boa visualização, basta digitar o nome do dataframe

In [ ]:
dados

,Grupo,Ângulo de abertura,Comprimento do fio,Instrumento de medida,Período de oscilação
0,Grupo 1,10.0,14.50,Régua,2.423
1,Grupo 2,NaN,14.45,Paquímetro,2.419
2,Grupo 3,20.0,14.50,NaN,2.437
3,Grupo 4,30.0,NaN,Paquímetro,2.451
4,Grupo 5,8.0,14.30,Régua,2.405
5,Grupo 6,15.0,14.50,Trena,2.429


## Anonimização dos dados
Para anonimizar os dados, basta remover a coluna de identificação dos grupos.

In [ ]:
dados_tratados = dados.drop(columns=['Grupo'])
dados_tratados

,Ângulo de abertura,Comprimento do fio,Instrumento de medida,Período de oscilação
0,10.0,14.50,Régua,2.423
1,NaN,14.45,Paquímetro,2.419
2,20.0,14.50,NaN,2.437
3,30.0,NaN,Paquímetro,2.451
4,8.0,14.30,Régua,2.405
5,15.0,14.50,Trena,2.429


## Trabalhando com dados ausentes
O tratamento dos dados ausentes deve ser feito caso a caso.
Para a característica "Instrumento de medida" faz sentido inserir um valor fixo. Neste caso, vamos adicionar o valor referente ao pior caso, como uma decisão conservadora.

In [ ]:
dados_tratados['Instrumento de medida'] = dados_tratados['Instrumento de medida'].fillna(value='Trena')
dados_tratados

,Ângulo de abertura,Comprimento do fio,Instrumento de medida,Período de oscilação
0,10.0,14.50,Régua,2.423
1,NaN,14.45,Paquímetro,2.419
2,20.0,14.50,Trena,2.437
3,30.0,NaN,Paquímetro,2.451
4,8.0,14.30,Régua,2.405
5,15.0,14.50,Trena,2.429


Para o caso da característica "Ângulo de abertura", o mais conservador é adotar a mediana dos valore presentes.

In [ ]:
dados_tratados["Ângulo de abertura"] = dados_tratados["Ângulo de abertura"].fillna(dados_tratados["Ângulo de abertura"].median())
dados_tratados

,Ângulo de abertura,Comprimento do fio,Instrumento de medida,Período de oscilação
0,10.0,14.50,Régua,2.423
1,15.0,14.45,Paquímetro,2.419
2,20.0,14.50,Trena,2.437
3,30.0,NaN,Paquímetro,2.451
4,8.0,14.30,Régua,2.405
5,15.0,14.50,Trena,2.429


Já para o caso do "Comprimento do fio", inserimos a média.

In [ ]:
dados_tratados['Comprimento do fio'] = dados_tratados['Comprimento do fio'].fillna(dados_tratados['Comprimento do fio'].mean())
dados_tratados

,Ângulo de abertura,Comprimento do fio,Instrumento de medida,Período de oscilação
0,10.0,14.50,Régua,2.423
1,15.0,14.45,Paquímetro,2.419
2,20.0,14.50,Trena,2.437
3,30.0,14.45,Paquímetro,2.451
4,8.0,14.30,Régua,2.405
5,15.0,14.50,Trena,2.429


## Transformação de dados
Um ponto importante é a cosntrução de atributos. Neste exemplo, vamos criar os atributos referentes aos "Instrumento de medida". Note que textos têm restrição nos algoritmos.
Vamos utilizar a ferramento OneHotEncoder da biblioteca do Scikit-Learn.

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
encoder = OneHotEncoder()
#encoder = OneHotEncoder(categories=[['Régua','Trena','Paquímetro']])
encoder.fit_transform(dados_tratados['Instrumento de medida'].values.reshape(-1,1))

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 6 stored elements and shape (6, 3)>

Aqui temos um problema. Você conseguiu identificar qual é?

In [ ]:
buffer = encoder.transform(dados_tratados['Instrumento de medida'].values.reshape(-1,1)).toarray()

In [ ]:
buffer

array([[0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

Transformando de volta em um Pandas DataFrame:

In [ ]:
dados_provisorio = pd.DataFrame(data=buffer, columns=['Régua','Trena','Paquímetro'])
dados_provisorio

,Régua,Trena,Paquímetro
0,0.0,1.0,0.0
1,1.0,0.0,0.0
2,0.0,0.0,1.0
3,1.0,0.0,0.0
4,0.0,1.0,0.0
5,0.0,0.0,1.0


Juntando o resultado no DataFrame original:

In [ ]:
dados_tratados = dados_tratados.join(dados_provisorio)
dados_tratados

,Ângulo de abertura,Comprimento do fio,Instrumento de medida,Período de oscilação,Régua,Trena,Paquímetro
0,10.0,14.50,Régua,2.423,0.0,1.0,0.0
1,15.0,14.45,Paquímetro,2.419,1.0,0.0,0.0
2,20.0,14.50,Trena,2.437,0.0,0.0,1.0
3,30.0,14.45,Paquímetro,2.451,1.0,0.0,0.0
4,8.0,14.30,Régua,2.405,0.0,1.0,0.0
5,15.0,14.50,Trena,2.429,0.0,0.0,1.0


Removendo dados redundantes:

In [ ]:
dados_tratados = dados_tratados.drop(columns=['Instrumento de medida'])
dados_tratados

,Ângulo de abertura,Comprimento do fio,Período de oscilação,Régua,Trena,Paquímetro
0,10.0,14.50,2.423,0.0,1.0,0.0
1,15.0,14.45,2.419,1.0,0.0,0.0
2,20.0,14.50,2.437,0.0,0.0,1.0
3,30.0,14.45,2.451,1.0,0.0,0.0
4,8.0,14.30,2.405,0.0,1.0,0.0
5,15.0,14.50,2.429,0.0,0.0,1.0


Agora vamos utilizar uma transformação de escala. Vamos utilizar a ferramenta MinMaxScaler da biblioteca do Scikit-Learn.

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
dados_tratados['Comprimento do fio'] = scaler.fit_transform(dados_tratados[['Comprimento do fio']])
dados_tratados

,Ângulo de abertura,Comprimento do fio,Período de oscilação,Régua,Trena,Paquímetro
0,10.0,1.00,2.423,0.0,1.0,0.0
1,15.0,0.75,2.419,1.0,0.0,0.0
2,20.0,1.00,2.437,0.0,0.0,1.0
3,30.0,0.75,2.451,1.0,0.0,0.0
4,8.0,0.00,2.405,0.0,1.0,0.0
5,15.0,1.00,2.429,0.0,0.0,1.0


Você consegue repetir o procedimento para a característica "Período de oscilação" e utilizando a ferramenta StandardScaler?

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
dados_tratados['Período de oscilação'] = scaler.fit_transform(dados_tratados[['Período de oscilação']])
dados_tratados


,Ângulo de abertura,Comprimento do fio,Período de oscilação,Régua,Trena,Paquímetro
0,10.0,1.00,-0.301026,0.0,1.0,0.0
1,15.0,0.75,-0.578896,1.0,0.0,0.0
2,20.0,1.00,0.671519,0.0,0.0,1.0
3,30.0,0.75,1.644065,1.0,0.0,0.0
4,8.0,0.00,-1.551441,0.0,1.0,0.0
5,15.0,1.00,0.115779,0.0,0.0,1.0
